<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Описание данных:**

Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.

Целевой признак: количество страховых выплат клиенту за последние 5 лет.

**Цель исследования**

Разработать метод преобразования данных, чтобы по нему было сложно восстановить персональную информацию, обосновать корректность его работы, при этом не допустить, чтобы при преобразовании качество моделей машинного обучения  ухудшилось. 

**Ход исследования**

Данные о тарифах получены их файла `datasets/insurance.csv`. В проекте мы изучим данные, определим возможность преобразования данных с защитой персональной информации, корректность ее работы.

Таким образом, исследование пройдет в ряд этапов:
- открытие файла и изучение данных; 
- анализ возможности ухудшения качества модели после преобразования, путем умножения исходной матрицы на обратимую;
- определение алгоритма преобразования данных;
- исследование данного алгоритма на предмет проверки на возможность ухудшения качества модели после преобразования данных. 
- формулировка итоговых выводов.

## Загрузка данных

Первоначально загрузим необходимые нам в работе библиотеки.

In [8]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

Загрузим датасет и взглянем на него.

In [9]:
try:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')
except:
    df = pd.read_csv('D:/learn_data/insurance.csv')

df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Посмотрим на общую информацию о датафрейме.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Как мы видим, в датасете имеется информация о 5000 клиентах, с указанием 4 признаков (пол, возраст, зарплата, количество членов семьи) и целевого признака (количество страховых выплат за последние 5 лет). Размер датасета составляет 195.4 Kb.

Далее взглянем на количество уникальных значений каждого признака, а также на количество возможных пропусков. 

In [11]:
for column in df.columns:
    print('Количество уникальных значений столбца', column, ':', len(df[column].unique()))
print()
print('Количество пропусков в столбце:')    
print(df.isna().mean())

Количество уникальных значений столбца Пол : 2
Количество уникальных значений столбца Возраст : 46
Количество уникальных значений столбца Зарплата : 524
Количество уникальных значений столбца Члены семьи : 7
Количество уникальных значений столбца Страховые выплаты : 6

Количество пропусков в столбце:
Пол                  0.0
Возраст              0.0
Зарплата             0.0
Члены семьи          0.0
Страховые выплаты    0.0
dtype: float64


Как мы видим, количество уникальны значений гораздо меньше, чем число объектов, что говорит о невозможности осуществления проверки на наличие дубликатов ввиду отсутствия данных, уникальных для каждого объекта. Пропуски отсутствуют.

Изменим тип данных в столбцах `'Возраст'` и `'Зарплата'` на целочисленный ввиду отсуствия дробных частей. Так же уменьшим обращение к памяти для остальных столбцов.

In [12]:
for column in df.columns:
    df[column] = pd.to_numeric(df[column], downcast='integer')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int8   
 1   Возраст            5000 non-null   int8   
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int8   
 4   Страховые выплаты  5000 non-null   int8   
dtypes: float64(1), int8(4)
memory usage: 58.7 KB


Таким образом, мы существенно уменьшили обращение датасета к памяти со 195.4 Kb до 58.7 Kb.

In [13]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


**Вывод:**

Данные загружены, не имеют пропусков. Изменение типа данных позволит нам быстрее обращаться к данным.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится.

**Обоснование:** 

Заменим матрицу $Х$ на $Z$, путем умножения матрицы $Х$ на некую квадратную обратимую матрицу $P$.

$$
Z = XP~~~~~(уравнение~1)
$$

Таким образом для матрицы $Z$ предсказание и вектор весов будет равно:

$$
a_1 = Zw_1~~~~~(уравнение~2)
$$
$$
w_1 = (Z^T Z)^{-1} Z^Ty~~~~~(уравнение~3)
$$

Подставим в уравнение 2 значение $w_1$ из уравнения 3

$$
a_1 = Z(Z^T Z)^{-1} Z^Ty
$$

Теперь заменим все $Z$ правой частью уравнения 1 и получим следующее

$$
a_1 = XP((XP)^T(XP))^{-1}(XP)^Ty
$$

Теперь используем свойство обратной матрицы, согласно которому обратное произведение матриц равно произведению обратных матриц, взятых в обратном порядке, то есть $(AB)^{-1} = B^{-1}A^{-1}$, вследствие чего, раскрыв через данное свойство часть уравнения $((XP)^T(XP))^{-1}$ получаем следующее

$$
a_1 = XP(XP)^{-1}((XP)^T)^{-1}(XP)^Ty
$$

Далее с помощью того же свойства обратной матрицы раскроем часть уравнения $(XP)^{-1}$, в результате чего получаем

$$
a_1 = XPP^{-1}X^{-1}((XP)^T)^{-1}(XP)^Ty
$$

Понимая, что $PP^{-1} = E$, то есть дает нам единичную матрицу, которая никак не влияет на результат уравнения и ее можно просто убрать, далее воспользуемся свойством транспонированной матрицы, согласно которому транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке, то есть $(AB)^T = B^TA^T$, раскроем $(XP)^T$ и получим

$$
a_1 = XEX^{-1}((XP)^T)^{-1}(XP)^Ty
$$
$$
a_1 = XX^{-1}(P^TX^T)^{-1}P^TX^Ty
$$

Затем вновь применим ранее упоминавшееся свойство обратной матрицы к части $(P^TX^T)^{-1}$ и получим следующее

$$
a_1 = XX^{-1}(X^T)^{-1}(P^T)^{-1}P^TX^Ty
$$

Учитывая, что часть уравнения $(P^T)^{-1}P^T = E$, то есть также дает нам единичную матрицу, взглянем на уравнение, применив свойство обратной матрицы к части уравнения $X^{-1}(X^T)^{-1}$

$$
a_1 = XX^{-1}(X^T)^{-1}EX^Ty
$$
$$
a_1 = X(X^TX)^{-1}X^Ty
$$

Как мы видим, осталось используя формулу предсказания ($a = Xw$) и формулу обучения ($w = (X^TX)^{-1}X^Ty$), заменить часть полученного уравнения $(X^TX)^{-1}X^Ty$ на $w$, чтобы доказать равенство $a$ и $a_1$, то есть равенство значений предсказаний как для оригинальной матрицы, так и для произведения такой матрицы на обратимую матрицу

$$
a_1 = X(X^TX)^{-1}X^Ty
$$
$$
a_1 = Xw
$$
$$
a_1 = a
$$

**Вывод:** 

Мы пришли к обоснованному выводу, что качество линейной регрессии для признаков, умноженных на обратимую матрицу, не изменится.

## Алгоритм преобразования

**Алгоритм**

Для защиты персональной информации будем умножать матрицу признаков на обратимую матрицу $P$, которая будет генерироваться случайным образом.

Этапы алгоритма:

1. Генерирование матрицы $P$ (размер матрицы выбираем в зависимости о количества признаков, то есть столбцов датасета, путем использования в коде shape[1]).
2. Проверка матрицы $P$ на обратимость путем вычисления определителя матрицы $P$ для соблюдения необходимости применения обратимой матрицы. В случае необратимости сгенерированной матрицы снова возвращаемся к п.1 алгоритма.
3. Получение матрицы преобразованных признаков $Z = XP$.
4. Применение алгоритма на преобразованных признаках $Z$.

**Обоснование**

Матрица $P$ должна иметь необходимую размерность (n x n), где n - количество признаков для регрессии. Таким образом матрица $Z$ будет иметь туже размерность, что и матрица $X$. Обратная матрица существует только для квадратных невырожденных матриц (то есть определитель которых не равен нулю).

Учитывая, что количество признаков датасета (исключая целевой признак) равняется 4, необходимая обратимая матрица должна иметь размерность 4 х 4.  

Умножение матрицы оригинальных признаков на обратимую матрицу соответствущей размерности, приведет персональные данные в труднопонимаемый для человека вид, при этом, как мы доказали на предыдущем этапе, будет возможно использование зашифрованных таким образом данных для обучения модели линейной регрессии, качество которой для признаков, умноженных на обратимую матрицу, не изменится.

**Вывод:**

Предложен алгоритм защиты персональных данных, который мы проверим далее. 

## Проверка алгоритма

Сначала разобьем датасет на части с признаками и целевым признаком.

In [14]:
features = df.drop("Страховые выплаты", axis=1)
target = df["Страховые выплаты"]

features.shape, target.shape

((5000, 4), (5000,))

Проверим, что линейная регрессия из scikit-learn вычисляет значения по формуле $w = (X^T X)^{-1} X^T y$.

In [15]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
w = np.linalg.inv(X.T @ X) @ X.T @ y
display(w[1:])
model = LinearRegression()
model.fit(features, target)
model.coef_

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

Как мы видим, у нас имеется полное совпадение коэффициентов регрессии.

Теперь исследуем качество моделей линейной регрессии на:
- оригинальных признаках;
- на отмасштабированных оригинальных признаках;
- на признаках после преобразования;
- на отмасштабированных признаках после преобразования.

Для начала разделим признаки на обучающую и тестовую выборки.

In [16]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=3301)

Теперь взглянем на показатели модели на оригинальных признаках.

In [17]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_original_df = r2_score(target_test, model.predict(features_test))
print("R2 =", R2_original_df)

R2 = 0.4092825730468017


Теперь посмотрим на метрику модели после масштабирования.

In [18]:
regressor = LinearRegression()
scaler = StandardScaler()
pipeline = Pipeline([("standard_scaler", scaler), ("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_original_df_scaled = r2_score(target_test, pipeline.predict(features_test))
print("R2 =", R2_original_df_scaled)

R2 = 0.4092825730468006


Как мы видим, качество регрессии на оригинальных признаках после масштабирования осталось прежним, незначительные расхождения после 15 знака после запятой отнесем к особенностям точности хранения дробных чисел в python.

Теперь проведем шифрование признаков, то есть преобразуем датасет с признаками, умножив его на псевдослучайно сгенерированную обратимую матрицу, которую используем как шифр. При этом сразу введем защиту от случайного создания необратимой матрицы (то есть такой, определитель которой равен 0).

In [19]:
n = features.shape[1]
np.random.seed(3301)
matrix_cipher = np.random.randint(1, 10, (n,n))
det = np.linalg.det(matrix_cipher)
while det == 0:
    matrix_cipher = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(matrix_cipher)
encrypted_features = features @ matrix_cipher

Давайте взглянем на то, как выглядят оригинальные данные, и на то, насколько от них отличаются зашифрованные данные, а также на матрицу-шифр.

In [12]:
print('Оригинальные данные признаков')
display(features.head())
print()
print('Зашифрованные данные признаков')
display(encrypted_features.head())
print()
print('Матрица-шифр')
matrix_cipher

Оригинальные данные признаков


,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600.0,1
1,0,46,38000.0,1
2,0,29,21000.0,0
3,0,21,41700.0,2
4,1,28,26100.0,0



Зашифрованные данные признаков


,0,1,2,3
0,49894.0,446700.0,446654.0,49942.0
1,38327.0,342326.0,342281.0,38373.0
2,21203.0,189203.0,189174.0,21232.0
3,41857.0,375455.0,375436.0,41878.0
4,26298.0,235105.0,235071.0,26333.0



Матрица-шифр


array([[2, 9, 3, 9],
       [7, 7, 6, 8],
       [1, 9, 9, 1],
       [5, 4, 5, 5]])

Как мы видим, в зашифрованных данных в качестве значений указаны совершенно непонятные числа. Шифрование прошло успешно. Теперь разделим зашифрованные данные на обучающую и тестовую выборки, и проверим R2-метрику модели, обученной на них. 

In [13]:
encrypted_features_train, encrypted_features_test, target_train, target_test = train_test_split(
    encrypted_features, target, test_size=0.25, random_state=3301)

In [14]:
model = LinearRegression()
model.fit(encrypted_features_train, target_train)
R2_cipher_df = r2_score(target_test, model.predict(encrypted_features_test))
print("R2 =", R2_cipher_df)

R2 = 0.40928257304664983


Теперь взглянем на R2-метрику модели, обученной на зашифрованных данных с применением к ним масштабирования.

In [15]:
regressor = LinearRegression()
scaler = StandardScaler()
pipeline = Pipeline([("standard_scaler", scaler),("linear_regression", regressor)])
pipeline.fit(encrypted_features_train, target_train)
R2_cipher_df_scaled = r2_score(target_test, pipeline.predict(encrypted_features_test))
print("R2 =", R2_cipher_df_scaled)

R2 = 0.40928257304680726


Окончательно сравним R2-метрики всех обученных моделей.

In [16]:
result = pd.DataFrame(data= [R2_original_df,
                      R2_original_df_scaled,
                      R2_cipher_df,
                      R2_cipher_df_scaled], 
                     columns=['R2'], 
                     index=['Линейная регрессия',
                            'Линейная регрессия c масштабированнием',
                            'Линейная регрессия на преобразованных признаках',
                            'Линейная регрессия на преобразованных признаках c масштабированием'])
result

,R2
Линейная регрессия,0.409283
Линейная регрессия c масштабированнием,0.409283
Линейная регрессия на преобразованных признаках,0.409283
Линейная регрессия на преобразованных признаках c масштабированием,0.409283


**Вывод:**

Все R2-метрики, как на оригинальных данных, так и на преобразованных данных, как с применением масштабирования, так и без этого, одинаковы. 

**Общие выводы:**

В ходе работы мы загрузили и изучили данные, определи, что качество линейной регрессии не меняется от использования оригнальной матрицы, так и матрицы, умноженной на обратимую, создали алгоритм преобразования данных, после чего исследовали алгоритм преобразования данных и проверили метрику R2 для данных без преобразования и с ним.

По результатам работы можно утверждать, что данные можно легко зашифровать от распознавания, имея правильную (обратимую и соответствующей размерности) матрицу.